In [ ]:
!pip install docxtpl

In [ ]:
def get_initials(full_name):
  names = full_name.split(" ")
  return (names[0])[0] + (names[len(names) - 1])[0]

In [ ]:
from docxtpl import DocxTemplate, InlineImage
from datetime import datetime
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE

import json

doc = DocxTemplate("/content/drive/MyDrive/cv_model.docx")
dados = open('/content/drive/MyDrive/model.json',)
dados = json.load(dados)

#STYLES
font_styles = doc.styles
font_charstyle = font_styles.add_style('internal_titles', WD_STYLE_TYPE.CHARACTER)
font_object = font_charstyle.font
font_object.underline = True
font_object.size = Pt(14)
font_object.name = 'Arial'

font_charstyle = font_styles.add_style('internal_text', WD_STYLE_TYPE.CHARACTER)
font_object = font_charstyle.font
font_object.size = Pt(10)
font_object.name = 'Arial'

# PICTURE
tables = doc.tables
tables[0].rows[0].cells[1]._element.clear_content()
img = tables[0].rows[0].cells[1].add_paragraph().add_run().add_picture('/content/drive/MyDrive/profile.jpg')

# WORK HISTORY
if dados["experiences"]:
  doc.add_paragraph().add_run("\n\nWORK HISTORY", style='internal_titles').bold = True
  for work in dados["experiences"]:
    table_experiencia = doc.add_table(rows = 5, cols = 2)
    table_experiencia.rows[0].cells[0].paragraphs[0].add_run(work["company"], style='internal_text').bold = True
    table_experiencia.rows[1].cells[0].paragraphs[0].add_run("Institutional affiliation", style='internal_text').bold = True
    table_experiencia.rows[2].cells[0].paragraphs[0].add_run(work["duration"], style='internal_text')
    table_experiencia.rows[2].cells[1].paragraphs[0].add_run("Placement:", style='internal_text').bold = True
    table_experiencia.rows[2].cells[1].paragraphs[0].add_run(work["title"], style='internal_text')
    if work["description"]:
      table_experiencia.rows[3].cells[0].paragraphs[0].add_run("Activities", style='internal_text').bold = True
      table_experiencia.rows[3].cells[1].paragraphs[0].add_run(work["description"], style='internal_text')

# SKILLS
if dados["skills"]:
  doc.add_paragraph().add_run("SKILLS", style='internal_titles').bold = True
  doc.add_paragraph().add_run("5 = Master, 4 = Expert, 3 = Proficient, 2 = Familiar, 1 = Beginner", style='internal_text').italic = True
  for skill in dados["skills"]:
    table_skill = doc.add_table(rows = 1, cols = 2)
    table_skill.rows[0].cells[0].paragraphs[0].add_run(skill["skill"], style='internal_text')
    table_skill.rows[0].cells[1].paragraphs[0].add_run("assessment = " + str(int(int("0" if skill["votes"] == "" else skill["votes"])/5 + 1)), style='internal_text')

# EDUCATION
if dados["educations"]:
  doc.add_paragraph().add_run("EDUCATION", style='internal_titles').bold = True
  for education in dados["educations"]:
    table_education = doc.add_table(rows = 2, cols = 3)
    table_education.rows[0].cells[0].paragraphs[0].add_run(education["duration"], style='internal_text')
    table_education.rows[0].cells[1].paragraphs[0].add_run(education["course"], style='internal_text').bold = True
    table_education.rows[1].cells[1].paragraphs[0].add_run(education["school"], style='internal_text')

# LANGUAGE SKILLS
if dados["languages"]:
  doc.add_paragraph().add_run("LANGUAGE SKILLS", style='internal_titles').bold = True
  for language in dados["languages"]:
    table_education = doc.add_table(rows = 1, cols = 2)
    table_education.rows[0].cells[0].paragraphs[0].add_run(language["language"], style='internal_text').bold = True
    table_education.rows[0].cells[1].paragraphs[0].add_run("Spoken (Intermediate), Written (Intermediate), Listening (Intermediate)", style='internal_text')

cvid = "CV-EN-{0}-{1}".format(datetime.today().strftime('%Y%m%d'), get_initials(dados["name"]))

context = {'work' : dados["work"] + '\n\n'+ dados["info"], 'age':'', 'locate': dados["locate"], 'CV_ID': cvid}

doc.render(context)

doc.save("/content/drive/MyDrive/profile.docx")